In [1]:
import numpy as np
import pandas as pd
import tqdm

import peyes
import analysis.utils as u

## Load Data

In [2]:
dataset = peyes.datasets.lund2013(directory=u.DATASETS_DIR, save=False, verbose=True)
dataset.head()

,trial_id,subject_id,stimulus_type,stimulus_name,t,x,y,pupil,pixel_size,viewer_distance,MN,RA
0,1,TH20,moving_dot,1,0.0,123.2532,22.6264,NaN,0.037824,67.0,1.0,1.0
1,1,TH20,moving_dot,1,2.0,123.5395,22.9064,NaN,0.037824,67.0,1.0,1.0
2,1,TH20,moving_dot,1,4.0,123.2230,21.9909,NaN,0.037824,67.0,1.0,1.0
3,1,TH20,moving_dot,1,6.0,123.1883,21.7740,NaN,0.037824,67.0,1.0,1.0
4,1,TH20,moving_dot,1,8.0,125.0540,21.1805,NaN,0.037824,67.0,1.0,1.0


In [3]:
VIEWER_DISTANCE_CM = dataset["viewer_distance"].unique()[0]
PIXEL_SIZE_CM = dataset["pixel_size"].unique()[0]

PIXEL_SIZE_DEG = peyes._utils.pixel_utils.pixels_to_visual_angle(1, VIEWER_DISTANCE_CM, PIXEL_SIZE_CM)
DEG_IN_PIXEL = peyes._utils.pixel_utils.visual_angle_to_pixels(1, VIEWER_DISTANCE_CM, PIXEL_SIZE_CM)

print(f"Viewer distance:\t\t{10 * VIEWER_DISTANCE_CM}mm")
print(f"Pixel size:\t\t\t\t{10 * PIXEL_SIZE_CM:.3f}mm")
print(f"Pixel size:\t\t\t\t{PIXEL_SIZE_DEG:.3f}° (DVA)")
print(f"1° (DVA) in pixels:\t\t{DEG_IN_PIXEL:.2f}px")

Viewer distance:		670.0mm
Pixel size:				0.378mm
Pixel size:				0.032° (DVA)
1° (DVA) in pixels:		30.92px


## Event-Features Statistics
### (Image Stimuli Only)

In [4]:
QUANTILES = [0.001, 0.01, 0.025, 0.05, 0.5, 0.95, 0.975, 0.99, 0.999]
EVENT_PROPORTION = 95   # 95% of events
FEATURE_CI = 95         # 95% confidence interval for feature statistic

def labels_to_events(dataframe: pd.DataFrame):
    trial_ids = dataframe[peyes.constants.TRIAL_ID_STR].unique()
    annotators = set(dataframe.columns).intersection(set([ann for annotators in u.DATASET_ANNOTATORS.values() for ann in annotators]))
    event_dict = {}
    for i, trial_id in tqdm.tqdm(enumerate(trial_ids), total=len(trial_ids)):
        trial_data = dataframe[dataframe["trial_id"] == trial_id]
        stim_type, stim_name = trial_data[[peyes.constants.STIMULUS_TYPE_STR, peyes.constants.STIMULUS_NAME_STR]].values[0]
        t = trial_data[peyes.constants.T].values
        x = trial_data[peyes.constants.X].values
        y = trial_data[peyes.constants.Y].values
        pupil = trial_data[peyes.constants.PUPIL].values
        ps = trial_data[peyes.constants.PIXEL_SIZE_STR].values[0]
        vd = trial_data[peyes.constants.VIEWER_DISTANCE_STR].values[0]
        for annotator in annotators:
            evnts = peyes.create_events(
                labels=trial_data[annotator].values,
                t=t, x=x, y=y, pupil=pupil, pixel_size=ps, viewer_distance=vd,
            )
            evnts = pd.Series(evnts, name=(trial_id, annotator))
            event_dict[(trial_id, stim_type, stim_name, annotator)] = evnts
    event_df = pd.DataFrame(event_dict).T.dropna(axis=0, how='all')
    event_df.index.names = [
        peyes.constants.TRIAL_ID_STR, peyes.constants.STIMULUS_TYPE_STR, peyes.constants.STIMULUS_NAME_STR, "annotator"
    ]
    return event_df


def events_df_to_series(events_df: pd.DataFrame, min_num_samples: int = 2) -> pd.Series:
    events_as_series = events_df.groupby(
        level=np.arange(events_df.index.nlevels).tolist()
    ).apply(
        lambda sub: pd.Series(sub.values.flatten()).dropna()
    )
    events_as_series = events_as_series[events_as_series.map(lambda x: x.num_samples >= min_num_samples)]
    return events_as_series

In [5]:
events = labels_to_events(dataset)
events_series = events_df_to_series(events, min_num_samples=2)

image_events = events.xs(peyes.constants.IMAGE_STR, level=peyes.constants.STIMULUS_TYPE_STR)
image_events_series = events_series.xs(peyes.constants.IMAGE_STR, level=peyes.constants.STIMULUS_TYPE_STR)

100%|██████████| 63/63 [00:00<00:00, 86.50it/s] 


#### Feature Quantiles
For each (label, rater) pair, we calculate the total number of events and their distribution of duration, dispersion, max velocity and min velocity.  
We present this data as a table specifying the quantiles for 0.1%, 1%, 2.5%, 5%, 50%, 95%, 97.5%, 99%, 99.9%

In [6]:
event_quantiles = {}

for label in peyes._DataModels.EventLabelEnum.EventLabelEnum:
    if label.value == 0:
        continue
    series = image_events_series[image_events_series.apply(lambda x: x.label == label.value)]
    
    # COUNTS
    counts = series.groupby(level="annotator").size().rename("")
    counts.loc["both"] = counts.sum()
    
    # DURATIONS
    dur_percentiles = pd.concat([
        series.groupby(level="annotator").apply(
            lambda sub_series: sub_series.apply(lambda event: event.duration).quantile(q)
        ).rename(f"{100 * q}%") for q in QUANTILES
    ], axis=1)
    dur_percentiles.loc["both"] = [series.apply(lambda event: event.duration).quantile(q) for q in QUANTILES]
    
    if label == peyes._DataModels.EventLabelEnum.EventLabelEnum.BLINK:
        stats = pd.concat([counts, dur_percentiles], keys=["counts", "durations"], axis=1)
        event_quantiles[label.name] = stats
        continue
    
    # DISPERSIONS
    disp_percentiles = pd.concat([
        series.groupby(level="annotator").apply(
            lambda sub_series: sub_series.apply(lambda event: event.dispersion).quantile(q)
        ).rename(f"{100 * q}%") for q in QUANTILES
    ], axis=1)
    disp_percentiles.loc["both"] = [series.apply(lambda event: event.dispersion).quantile(q) for q in QUANTILES]
    
    # PEAK VELOCITIES
    peak_vel_percentiles = pd.concat([
        series.groupby(level="annotator").apply(
            lambda sub_series: sub_series.apply(lambda event: event.peak_velocity).quantile(q)
        ).rename(f"{100 * q}%") for q in QUANTILES
    ], axis=1)
    peak_vel_percentiles.loc["both"] = [series.apply(lambda event: event.peak_velocity).quantile(q) for q in QUANTILES]
    
    # MIN VELOCITIES
    min_vel_percentiles = pd.concat([
        series.groupby(level="annotator").apply(
            lambda sub_series: sub_series.apply(lambda event: event.min_velocity).quantile(q)
        ).rename(f"{100 * q}%") for q in QUANTILES
    ], axis=1)
    min_vel_percentiles.loc["both"] = [series.apply(lambda event: event.min_velocity).quantile(q) for q in QUANTILES]
    
    stats = pd.concat(
        [counts, dur_percentiles, disp_percentiles, peak_vel_percentiles, min_vel_percentiles],
        keys=["counts", "durations", "dispersions", "peak_velocities", "min_velocities"],
        axis=1
    )
    event_quantiles[label.name] = stats
    
event_quantiles = pd.concat(event_quantiles, axis=0, names=[peyes.constants.EVENT_STR, "annotator"])

print("##  IMPORTANT NOTE  ##")
print("Not all features are applicable to all events (e.g., blinks have no velocity), so some statistics may be missing.")
event_quantiles

##  IMPORTANT NOTE  ##
Not all features are applicable to all events (e.g., blinks have no velocity), so some statistics may be missing.


counts   durations                                    \
                                       0.1%       1.0%        2.5%       5.0%   
event          annotator                                                        
FIXATION       MN           404    6.809030   32.43285   78.308300   96.01135   
               RA           563   13.867562   50.48652   80.218900  100.02150   
               both         967    5.999000   46.00866   80.011350   96.02080   
SACCADE        MN           377    6.003632   10.00028   10.010000   12.00000   
               RA           552    8.000551    9.99608   11.998100   12.01540   
               both         929    6.007496    9.99856   10.011000   12.00240   
PSO            MN           312    3.995244    5.99811    6.000000    7.99620   
               RA           418    4.001000    4.99817    6.004700    7.99900   
               both         730    3.996916    4.99829    6.001000    7.99900   
SMOOTH_PURSUIT MN             3  152.582114  157.40714  165.448850  178.85170   
               RA            30   54.529464   59.23164   67.068600  100.83020   
               both          33   54.583512   59.77212   68.419800  110.43260   
BLINK          MN            22   90.129727   91.07227   92.643175   97.86260   
               RA            23   94.046714   94.24214   94.567850   99.51400   
               both          45   90.201000   91.78500   94.123700   95.01200   

                                                                       \
                             50.0%      95.0%       97.5%       99.0%   
event          annotator                                                
FIXATION       MN         220.0500  521.03995  681.388700  1173.03150   
               RA         226.0500  490.70040  603.061150  1075.83428   
               both       224.9990  508.56940  626.323700  1160.43630   
SACCADE        MN          28.0010   52.00820   57.213200    66.01340   
               RA          30.0175   55.45190   60.012825    72.44506   
               both        30.0020   54.00960   60.008800    68.88444   
PSO            MN          21.9990   50.00945   56.916825    64.45832   
               RA          19.9985   40.01015   50.004025    55.00066   
               both        20.0050   46.00855   54.020625    60.00000   
SMOOTH_PURSUIT MN         420.1030  502.90570  507.505850   510.26594   
               RA         268.0605  658.93170  691.294600   702.61504   
               both       270.0600  650.52960  689.342800   701.83432   
BLINK          MN         319.0690  520.01890  574.276600   610.19164   
               RA         329.9960  623.73790  839.885800   993.70012   
               both       322.0710  617.33680  633.735400   892.91548   

                                       ... peak_velocities min_velocities  \
                                99.9%  ...           99.9%           0.1%   
event          annotator               ...                                  
FIXATION       MN         3530.437680  ...      170.529352       0.035566   
               RA         3175.250348  ...      164.736555       0.036727   
               both       4421.096796  ...      170.223810       0.032622   
SACCADE        MN          110.507248  ...      171.630742       1.401812   
               RA          107.196919  ...      171.702116       1.740005   
               both        116.173856  ...      172.872905       1.658814   
PSO            MN           69.998689  ...      151.529737       1.357170   
               RA           65.830583  ...      145.134590       0.679208   
               both         69.999542  ...      148.663984       0.773734   
SMOOTH_PURSUIT MN          511.921994  ...      161.692930       1.456277   
               RA          709.407304  ...      169.462619       0.371445   
               both        709.329232  ...      169.438952       0.371848   
BLINK          MN          631.740664  ...             NaN            NaN   
               RA  

In [30]:
event_quantiles.xs('durations', axis=1)

0.1%       1.0%        2.5%       5.0%  \
event          annotator                                                 
FIXATION       MN           6.809030   32.43285   78.308300   96.01135   
               RA          13.867562   50.48652   80.218900  100.02150   
               both         5.999000   46.00866   80.011350   96.02080   
SACCADE        MN           6.003632   10.00028   10.010000   12.00000   
               RA           8.000551    9.99608   11.998100   12.01540   
               both         6.007496    9.99856   10.011000   12.00240   
PSO            MN           3.995244    5.99811    6.000000    7.99620   
               RA           4.001000    4.99817    6.004700    7.99900   
               both         3.996916    4.99829    6.001000    7.99900   
SMOOTH_PURSUIT MN         152.582114  157.40714  165.448850  178.85170   
               RA          54.529464   59.23164   67.068600  100.83020   
               both        54.583512   59.77212   68.419800  110.43260   
BLINK          MN          90.129727   91.07227   92.643175   97.86260   
               RA          94.046714   94.24214   94.567850   99.51400   
               both        90.201000   91.78500   94.123700   95.01200   

                             50.0%      95.0%       97.5%       99.0%  \
event          annotator                                                
FIXATION       MN         220.0500  521.03995  681.388700  1173.03150   
               RA         226.0500  490.70040  603.061150  1075.83428   
               both       224.9990  508.56940  626.323700  1160.43630   
SACCADE        MN          28.0010   52.00820   57.213200    66.01340   
               RA          30.0175   55.45190   60.012825    72.44506   
               both        30.0020   54.00960   60.008800    68.88444   
PSO            MN          21.9990   50.00945   56.916825    64.45832   
               RA          19.9985   40.01015   50.004025    55.00066   
               both        20.0050   46.00855   54.020625    60.00000   
SMOOTH_PURSUIT MN         420.1030  502.90570  507.505850   510.26594   
               RA         268.0605  658.93170  691.294600   702.61504   
               both       270.0600  650.52960  689.342800   701.83432   
BLINK          MN         319.0690  520.01890  574.276600   610.19164   
               RA         329.9960  623.73790  839.885800   993.70012   
               both       322.0710  617.33680  633.735400   892.91548   

                                99.9%  
event          annotator               
FIXATION       MN         3530.437680  
               RA         3175.250348  
               both       4421.096796  
SACCADE        MN          110.507248  
               RA          107.196919  
               both        116.173856  
PSO            MN           69.998689  
               RA           65.830583  
               both         69.999542  
SMOOTH_PURSUIT MN          511.921994  
               RA          709.407304  
               both        709.329232  
BLINK          MN          631.740664  
               RA         1085.988712  
               both       1075.910248

#### Threshold Cutoffs
We specify the % of events that pass the selected thresholds (see table of thresholds)

In [31]:
thresholds = pd.DataFrame.from_dict({
    1: {
        ('duration', 'min'): 55,        # used by Andersson et. al (2016)
        ('duration', 'max'): 5000,
        ('dispersion', 'max'): 2.7      # used by Andersson et. al (2016) - I-DT
    },
    2: {
        ('duration', 'min'): 6,
        ('duration', 'max'): 150,
        ('dispersion', 'min'): 2.7,     # used by Andersson et. al (2016) - I-DT
        ('min_velocity', 'min'): 45     # used by Andersson et. al (2016) - I-VT
    },
    3: {
        ('duration', 'min'): 2,
        ('duration', 'max'): 80,
    },
    4: {
        ('duration', 'min'): 55,
        ('duration', 'max'): 5000,
        ('dispersion', 'min'): 2.7,     # used by Andersson et. al (2016) - I-DT
        ('min_velocity', 'min'): 26     # used by Komogortsev & Karpov (2013) - I-VVT
    },
    5: {
        ('duration', 'min'): 50,
        ('duration', 'max'): 2000,
    },
}).sort_index().T
thresholds

dispersion      duration       min_velocity
         max  min      max   min          min
1        2.7  NaN   5000.0  55.0          NaN
2        NaN  2.7    150.0   6.0         45.0
3        NaN  NaN     80.0   2.0          NaN
4        NaN  2.7   5000.0  55.0         26.0
5        NaN  NaN   2000.0  50.0          NaN

In [32]:
success_rates = {}

for idx in sorted(thresholds.index.unique()):
    label = peyes.parse_label(idx)
    series = image_events_series[image_events_series.apply(lambda x: x.label == idx)]
    counts = series.groupby(level="annotator").size().rename("total")
    counts.loc["both"] = counts.sum()
    success_rates[(label.name, "counts", '')] = counts
    
    for feat, op in thresholds.columns:
        if np.isnan(thresholds.loc[idx, (feat, op)]):
            continue
        if op == "min":
            success = series.apply(lambda event: getattr(event, feat) >= thresholds.loc[idx, (feat, op)])
        elif op == "max":
            success = series.apply(lambda event: getattr(event, feat) <= thresholds.loc[idx, (feat, op)])
        else:
            raise ValueError(f"Invalid operation: {op}")
        success = success.groupby(level="annotator").sum()
        success.loc["both"] = success.sum()
        success_rates[(label.name, feat, op)] = (100 * success / counts).rename((feat, op))

success_rates = pd.concat(success_rates, axis=1).stack(0, future_stack=True).reorder_levels([1, 0])
success_rates = success_rates.reindex(event_quantiles.index)
success_rates = success_rates[sorted(success_rates.columns)]

print("The following values show what % of events meet the thresholds specified in the above code-box.")
success_rates

The following values show what % of events meet the thresholds specified in the above code-box.


counts dispersion            duration              \
                                       max        min      max         min   
event          annotator                                                     
FIXATION       MN           404  98.019802        NaN    100.0   98.019802   
               RA           563  99.111901        NaN    100.0   98.756661   
               both         967  98.655636        NaN    100.0   98.448811   
SACCADE        MN           377        NaN  79.575597    100.0  100.000000   
               RA           552        NaN  79.166667    100.0  100.000000   
               both         929        NaN  79.332616    100.0  100.000000   
PSO            MN           312        NaN        NaN    100.0  100.000000   
               RA           418        NaN        NaN    100.0  100.000000   
               both         730        NaN        NaN    100.0  100.000000   
SMOOTH_PURSUIT MN             3        NaN  66.666667    100.0  100.000000   
               RA            30        NaN  20.000000    100.0   96.666667   
               both          33        NaN  24.242424    100.0   96.969697   
BLINK          MN            22        NaN        NaN    100.0  100.000000   
               RA            23        NaN        NaN    100.0  100.000000   
               both          45        NaN        NaN    100.0  100.000000   

                         min_velocity  
                                  min  
event          annotator               
FIXATION       MN                 NaN  
               RA                 NaN  
               both               NaN  
SACCADE        MN           27.055703  
               RA           20.833333  
               both         23.358450  
PSO            MN                 NaN  
               RA                 NaN  
               both               NaN  
SMOOTH_PURSUIT MN            0.000000  
               RA            0.000000  
               both          0.000000  
BLINK          MN                 NaN  
               RA                 NaN  
               both               NaN

========================================================================================================================

### Fixation Features

In [10]:
fix_series = image_events_series[image_events_series.apply(lambda x: x.label == 1)]
fix_counts = fix_series.groupby(level="annotator").size().rename("total")

**DURATIONS**

In [11]:
fix_dur_percentiles = [
    fix_series.groupby(level="annotator").apply(
        lambda sub_series: sub_series.apply(lambda fix: fix.duration).quantile(q)
    ).rename(f"{100*q}%") for q in QUANTILES
]
fix_dur_percentiles = pd.concat([fix_counts, *fix_dur_percentiles], axis=1)
fix_dur_percentiles.loc["both"] = [
    fix_counts.sum(),
    *[fix_series.apply(lambda fix: fix.duration).quantile(q) for q in QUANTILES]
]

print("FIXATION DURATIONS (ms)")
print(f"{EVENT_PROPORTION}% have duration >= {fix_dur_percentiles.loc['both', '5.0%']:.2f}ms")
print(f"{EVENT_PROPORTION}% have duration <= {fix_dur_percentiles.loc['both', '95.0%']:.2f}ms")
print(f"{FEATURE_CI}% CI for duration is [{fix_dur_percentiles.loc['both', '2.5%']:.2f}, {fix_dur_percentiles.loc['both', '97.5%']:.2f}]ms\n")
fix_dur_percentiles

FIXATION DURATIONS (ms)
95% have duration >= 96.02ms
95% have duration <= 508.57ms
95% CI for duration is [80.01, 626.32]ms



,total,0.1%,1.0%,2.5%,5.0%,50.0%,95.0%,97.5%,99.0%,99.9%
annotator,,,,,,,,,,
MN,404.0,6.809030,32.43285,78.30830,96.01135,220.050,521.03995,681.38870,1173.03150,3530.437680
RA,563.0,13.867562,50.48652,80.21890,100.02150,226.050,490.70040,603.06115,1075.83428,3175.250348
both,967.0,5.999000,46.00866,80.01135,96.02080,224.999,508.56940,626.32370,1160.43630,4421.096796


**DISPERSIONS**

In [12]:
fix_disp_percentiles = [
    fix_series.groupby(level="annotator").apply(
        lambda sub_series: sub_series.apply(lambda fix: fix.dispersion).quantile(q)
    ).rename(f"{100*q}%") for q in QUANTILES
]
fix_disp_percentiles = pd.concat([fix_counts, *fix_disp_percentiles], axis=1)
fix_disp_percentiles.loc["both"] = [
    fix_counts.sum(),
    *[fix_series.apply(lambda fix: fix.dispersion).quantile(q) for q in QUANTILES]
]

print("FIXATION DISPERSIONS (DVA)")
print(f"{EVENT_PROPORTION}% have dispersion <= {fix_disp_percentiles.loc['both', '95.0%']:.2f}°")
print(f"{FEATURE_CI}% CI for dispersion is [{fix_disp_percentiles.loc['both', '2.5%']:.2f}, {fix_disp_percentiles.loc['both', '97.5%']:.2f}]°\n")
fix_disp_percentiles

FIXATION DISPERSIONS (DVA)
95% have dispersion <= 1.71°
95% CI for dispersion is [0.33, 2.08]°



,total,0.1%,1.0%,2.5%,5.0%,50.0%,95.0%,97.5%,99.0%,99.9%
annotator,,,,,,,,,,
MN,404.0,0.107366,0.264291,0.313596,0.427110,0.889216,1.845093,2.443368,3.583134,4.307582
RA,563.0,0.189296,0.276676,0.361741,0.415948,0.850519,1.562763,1.862949,2.368435,5.755802
both,967.0,0.180766,0.266612,0.333404,0.418469,0.862999,1.712049,2.083268,3.244625,4.455510


**PEAK VELOCITIES**

In [13]:
fix_peak_vel_percentiles = [
    fix_series.groupby(level="annotator").apply(
        lambda sub_series: sub_series.apply(lambda fix: fix.peak_velocity).quantile(q)
    ).rename(f"{100*q}%") for q in QUANTILES
]
fix_peak_vel_percentiles = pd.concat([fix_counts, *fix_peak_vel_percentiles], axis=1)
fix_peak_vel_percentiles.loc["both"] = [
    fix_counts.sum(),
    *[fix_series.apply(lambda fix: fix.peak_velocity).quantile(q) for q in QUANTILES]
]

print("FIXATION PEAK VELOCITIES (DVA/s)")
print(f"{EVENT_PROPORTION}% have peak velocity <= {fix_peak_vel_percentiles.loc['both', '95.0%']:.2f}°/s")
print(f"{FEATURE_CI}% CI for peak velocity is [{fix_peak_vel_percentiles.loc['both', '2.5%']:.2f}, {fix_peak_vel_percentiles.loc['both', '97.5%']:.2f}]°/s\n")
fix_peak_vel_percentiles

FIXATION PEAK VELOCITIES (DVA/s)
95% have peak velocity <= 81.41°/s
95% CI for peak velocity is [10.64, 93.40]°/s



,total,0.1%,1.0%,2.5%,5.0%,50.0%,95.0%,97.5%,99.0%,99.9%
annotator,,,,,,,,,,
MN,404.0,7.028247,9.195111,10.474831,13.254122,35.776222,86.751270,99.981131,153.501711,170.529352
RA,563.0,7.262858,9.396542,10.729934,14.056459,34.696983,77.840417,88.895023,135.511126,164.736555
both,967.0,6.786223,9.311124,10.639351,13.769389,35.051962,81.412127,93.401083,153.729007,170.223810


### Saccade Features

In [14]:
sac_series = image_events_series[image_events_series.apply(lambda x: x.label == 2)]
sac_counts = sac_series.groupby(level="annotator").size().rename("total")

**DURATIONS**

In [15]:
sac_dur_percentiles = [
    sac_series.groupby(level="annotator").apply(
        lambda sub_series: sub_series.apply(lambda sac: sac.duration).quantile(q)
    ).rename(f"{100*q}%") for q in QUANTILES
]
sac_dur_percentiles = pd.concat([sac_counts, *sac_dur_percentiles], axis=1)
sac_dur_percentiles.loc["both"] = [
    sac_counts.sum(),
    *[sac_series.apply(lambda sac: sac.duration).quantile(q) for q in QUANTILES]
]

print("SACCADE DURATIONS (ms)")
print(f"{EVENT_PROPORTION}% have duration >= {sac_dur_percentiles.loc['both', '5.0%']:.2f}ms")
print(f"{EVENT_PROPORTION}% have duration <= {sac_dur_percentiles.loc['both', '95.0%']:.2f}ms")
print(f"{FEATURE_CI}% CI for duration is [{sac_dur_percentiles.loc['both', '2.5%']:.2f}, {sac_dur_percentiles.loc['both', '97.5%']:.2f}]ms\n")
sac_dur_percentiles

SACCADE DURATIONS (ms)
95% have duration >= 12.00ms
95% have duration <= 54.01ms
95% CI for duration is [10.01, 60.01]ms



,total,0.1%,1.0%,2.5%,5.0%,50.0%,95.0%,97.5%,99.0%,99.9%
annotator,,,,,,,,,,
MN,377.0,6.003632,10.00028,10.0100,12.0000,28.0010,52.0082,57.213200,66.01340,110.507248
RA,552.0,8.000551,9.99608,11.9981,12.0154,30.0175,55.4519,60.012825,72.44506,107.196919
both,929.0,6.007496,9.99856,10.0110,12.0024,30.0020,54.0096,60.008800,68.88444,116.173856


**DISPERSIONS**

In [16]:
sac_disp_percentiles = [
    sac_series.groupby(level="annotator").apply(
        lambda sub_series: sub_series.apply(lambda sac: sac.dispersion).quantile(q)
    ).rename(f"{100*q}%") for q in QUANTILES
]
sac_disp_percentiles = pd.concat([sac_counts, *sac_disp_percentiles], axis=1)
sac_disp_percentiles.loc["both"] = [
    sac_counts.sum(),
    *[sac_series.apply(lambda sac: sac.dispersion).quantile(q) for q in QUANTILES]
]

print("SACCADE DISPERSIONS (DVA)")
print(f"{EVENT_PROPORTION}% have dispersion >= {sac_disp_percentiles.loc['both', '5.0%']:.2f}°")
print(f"{FEATURE_CI}% CI for dispersion is [{sac_disp_percentiles.loc['both', '2.5%']:.2f}, {sac_disp_percentiles.loc['both', '97.5%']:.2f}]°\n")
sac_disp_percentiles

SACCADE DISPERSIONS (DVA)
95% have dispersion >= 1.05°
95% CI for dispersion is [0.66, 20.15]°



,total,0.1%,1.0%,2.5%,5.0%,50.0%,95.0%,97.5%,99.0%,99.9%
annotator,,,,,,,,,,
MN,377.0,0.364736,0.537610,0.681484,1.055732,6.086238,16.512912,19.179137,20.978137,28.087049
RA,552.0,0.337743,0.465576,0.659310,1.056474,6.230289,16.717128,20.384894,24.734837,32.062907
both,929.0,0.338789,0.477715,0.664135,1.054093,6.123734,16.544584,20.148755,24.366417,30.959088


**MIN VELOCITIES**

In [17]:
sac_min_vel_percentiles = [
    sac_series.groupby(level="annotator").apply(
        lambda sub_series: sub_series.apply(lambda sac: sac.min_velocity).quantile(q)
    ).rename(f"{100*q}%") for q in QUANTILES
]
sac_min_vel_percentiles = pd.concat([sac_counts, *sac_min_vel_percentiles], axis=1)
sac_min_vel_percentiles.loc["both"] = [
    sac_counts.sum(),
    *[sac_series.apply(lambda sac: sac.min_velocity).quantile(q) for q in QUANTILES]
]

print("SACCADE MIN VELOCITIES (DVA/s)")
print(f"{EVENT_PROPORTION}% have min velocity >= {sac_min_vel_percentiles.loc['both', '5.0%']:.2f}°/s")
print(f"{FEATURE_CI}% CI for min velocity is [{sac_min_vel_percentiles.loc['both', '2.5%']:.2f}, {sac_min_vel_percentiles.loc['both', '97.5%']:.2f}]°/s\n")
sac_min_vel_percentiles

SACCADE MIN VELOCITIES (DVA/s)
95% have min velocity >= 6.72°/s
95% CI for min velocity is [4.50, 93.29]°/s



,total,0.1%,1.0%,2.5%,5.0%,50.0%,95.0%,97.5%,99.0%,99.9%
annotator,,,,,,,,,,
MN,377.0,1.401812,2.436304,5.208581,7.240570,29.913226,84.466290,96.432994,106.657180,132.842196
RA,552.0,1.740005,3.078073,4.455919,6.470810,27.161543,76.615685,89.692423,104.063074,131.803931
both,929.0,1.658814,2.640067,4.502300,6.718161,28.440431,80.263971,93.290203,105.309931,138.093096


**PEAK VELOCITIES**

In [18]:
sac_peak_vel_percentiles = [
    sac_series.groupby(level="annotator").apply(
        lambda sub_series: sub_series.apply(lambda sac: sac.peak_velocity).quantile(q)
    ).rename(f"{100*q}%") for q in QUANTILES
]
sac_peak_vel_percentiles = pd.concat([sac_counts, *sac_peak_vel_percentiles], axis=1)
sac_peak_vel_percentiles.loc["both"] = [
    sac_counts.sum(),
    *[sac_series.apply(lambda sac: sac.peak_velocity).quantile(q) for q in QUANTILES]
]

print("SACCADE PEAK VELOCITIES (DVA/s)")
print(f"{EVENT_PROPORTION}% have peak velocity >= {sac_peak_vel_percentiles.loc['both', '5.0%']:.2f}°/s")
print(f"{EVENT_PROPORTION}% have peak velocity <= {sac_peak_vel_percentiles.loc['both', '95.0%']:.2f}°/s")
print(f"{FEATURE_CI}% CI for peak velocity is [{sac_peak_vel_percentiles.loc['both', '2.5%']:.2f}, {sac_peak_vel_percentiles.loc['both', '97.5%']:.2f}]°/s\n")
sac_peak_vel_percentiles

SACCADE PEAK VELOCITIES (DVA/s)
95% have peak velocity >= 86.21°/s
95% have peak velocity <= 161.95°/s
95% CI for peak velocity is [63.43, 163.22]°/s



,total,0.1%,1.0%,2.5%,5.0%,50.0%,95.0%,97.5%,99.0%,99.9%
annotator,,,,,,,,,,
MN,377.0,48.127007,52.213185,65.629423,88.197704,149.392146,161.970278,163.219300,164.701527,171.630742
RA,552.0,20.217231,47.881535,62.928936,85.042826,149.220219,161.881993,163.192786,164.436359,171.702116
both,929.0,23.736425,49.332073,63.429464,86.205582,149.299810,161.952677,163.218008,164.440961,172.872905


**AMPLITUDES**

In [19]:
sac_amp_percentiles = [
    sac_series.groupby(level="annotator").apply(
        lambda sub_series: sub_series.apply(lambda sac: sac.amplitude).quantile(q)
    ).rename(f"{100*q}%") for q in QUANTILES
]
sac_amp_percentiles = pd.concat([sac_counts, *sac_amp_percentiles], axis=1)
sac_amp_percentiles.loc["both"] = [
    sac_counts.sum(),
    *[sac_series.apply(lambda sac: sac.amplitude).quantile(q) for q in QUANTILES]
]

print("SACCADE AMPLITUDES (DVA)")
print(f"{EVENT_PROPORTION}% have amplitude >= {sac_amp_percentiles.loc['both', '5.0%']:.2f}°")
print(f"{EVENT_PROPORTION}% have amplitude <= {sac_amp_percentiles.loc['both', '95.0%']:.2f}°")
print(f"{FEATURE_CI}% CI for amplitude is [{sac_amp_percentiles.loc['both', '2.5%']:.2f}, {sac_amp_percentiles.loc['both', '97.5%']:.2f}]°\n")
sac_amp_percentiles

SACCADE AMPLITUDES (DVA)
95% have amplitude >= 0.76°
95% have amplitude <= 13.74°
95% CI for amplitude is [0.54, 16.24]°



,total,0.1%,1.0%,2.5%,5.0%,50.0%,95.0%,97.5%,99.0%,99.9%
annotator,,,,,,,,,,
MN,377.0,0.195701,0.466191,0.550604,0.806500,4.797977,13.093571,15.620893,17.831548,22.211912
RA,552.0,0.212589,0.392833,0.512532,0.758933,4.929352,14.151497,16.504438,20.743021,24.037944
both,929.0,0.174588,0.409880,0.538674,0.763880,4.893781,13.744142,16.239571,19.752714,23.388629


### Smooth Pursuit Features

In [20]:
sp_series = image_events_series[image_events_series.apply(lambda x: x.label == 4)]
sp_counts = sp_series.groupby(level="annotator").size().rename("total")

**DURATIONS**

In [21]:
sp_dur_percentiles = [
    sp_series.groupby(level="annotator").apply(
        lambda sub_series: sub_series.apply(lambda sp: sp.duration).quantile(q)
    ).rename(f"{100*q}%") for q in QUANTILES
]
sp_dur_percentiles = pd.concat([sp_counts, *sp_dur_percentiles], axis=1)
sp_dur_percentiles.loc["both"] = [
    sp_counts.sum(),
    *[sp_series.apply(lambda sp: sp.duration).quantile(q) for q in QUANTILES]
]

print("SMOOTH PURSUIT DURATIONS (ms)")
print(f"{EVENT_PROPORTION}% have duration >= {sp_dur_percentiles.loc['both', '5.0%']:.2f}ms")
print(f"{EVENT_PROPORTION}% have duration <= {sp_dur_percentiles.loc['both', '95.0%']:.2f}ms")
print(f"{FEATURE_CI}% CI for duration is [{sp_dur_percentiles.loc['both', '2.5%']:.2f}, {sp_dur_percentiles.loc['both', '97.5%']:.2f}]ms\n")
sp_dur_percentiles

SMOOTH PURSUIT DURATIONS (ms)
95% have duration >= 110.43ms
95% have duration <= 650.53ms
95% CI for duration is [68.42, 689.34]ms



,total,0.1%,1.0%,2.5%,5.0%,50.0%,95.0%,97.5%,99.0%,99.9%
annotator,,,,,,,,,,
MN,3.0,152.582114,157.40714,165.44885,178.8517,420.1030,502.9057,507.50585,510.26594,511.921994
RA,30.0,54.529464,59.23164,67.06860,100.8302,268.0605,658.9317,691.29460,702.61504,709.407304
both,33.0,54.583512,59.77212,68.41980,110.4326,270.0600,650.5296,689.34280,701.83432,709.329232


**MIN VELOCITIES**

In [22]:
sp_min_vel_percentiles = [
    sp_series.groupby(level="annotator").apply(
        lambda sub_series: sub_series.apply(lambda sp: sp.min_velocity).quantile(q)
    ).rename(f"{100*q}%") for q in QUANTILES
]
sp_min_vel_percentiles = pd.concat([sp_counts, *sp_min_vel_percentiles], axis=1)
sp_min_vel_percentiles.loc["both"] = [
    sp_counts.sum(),
    *[sp_series.apply(lambda sp: sp.min_velocity).quantile(q) for q in QUANTILES]
]

print("SMOOTH PURSUIT MIN VELOCITIES (DVA/s)")
print(f"{EVENT_PROPORTION}% have min velocity >= {sp_min_vel_percentiles.loc['both', '5.0%']:.2f}°/s")
print(f"{EVENT_PROPORTION}% have min velocity <= {sp_min_vel_percentiles.loc['both', '95.0%']:.2f}°/s")
print(f"{FEATURE_CI}% CI for min velocity is [{sp_min_vel_percentiles.loc['both', '2.5%']:.2f}, {sp_min_vel_percentiles.loc['both', '97.5%']:.2f}]°/s\n")
sp_min_vel_percentiles

SMOOTH PURSUIT MIN VELOCITIES (DVA/s)
95% have min velocity >= 0.54°/s
95% have min velocity <= 7.25°/s
95% CI for min velocity is [0.48, 8.59]°/s



,total,0.1%,1.0%,2.5%,5.0%,50.0%,95.0%,97.5%,99.0%,99.9%
annotator,,,,,,,,,,
MN,3.0,1.456277,1.467460,1.486098,1.517162,2.076302,2.488836,2.511755,2.525506,2.533757
RA,30.0,0.371445,0.406507,0.464942,0.529556,1.172636,7.388151,8.877995,10.575343,11.593751
both,33.0,0.371848,0.410537,0.475017,0.538780,1.220562,7.249214,8.585349,10.458284,11.582045


**PEAK VELOCITIES**

In [23]:
sp_peak_vel_percentiles = [
    sp_series.groupby(level="annotator").apply(
        lambda sub_series: sub_series.apply(lambda sp: sp.peak_velocity).quantile(q)
    ).rename(f"{100*q}%") for q in QUANTILES
]
sp_peak_vel_percentiles = pd.concat([sp_counts, *sp_peak_vel_percentiles], axis=1)
sp_peak_vel_percentiles.loc["both"] = [
    sp_counts.sum(),
    *[sp_series.apply(lambda sp: sp.peak_velocity).quantile(q) for q in QUANTILES]
]

print("SMOOTH PURSUIT PEAK VELOCITIES (DVA/s)")
print(f"{EVENT_PROPORTION}% have peak velocity >= {sp_peak_vel_percentiles.loc['both', '5.0%']:.2f}°/s")
print(f"{EVENT_PROPORTION}% have peak velocity <= {sp_peak_vel_percentiles.loc['both', '95.0%']:.2f}°/s")
print(f"{FEATURE_CI}% CI for peak velocity is [{sp_peak_vel_percentiles.loc['both', '2.5%']:.2f}, {sp_peak_vel_percentiles.loc['both', '97.5%']:.2f}]°/s\n")
sp_peak_vel_percentiles

SMOOTH PURSUIT PEAK VELOCITIES (DVA/s)
95% have peak velocity >= 30.90°/s
95% have peak velocity <= 161.80°/s
95% CI for peak velocity is [28.18, 163.38]°/s



,total,0.1%,1.0%,2.5%,5.0%,50.0%,95.0%,97.5%,99.0%,99.9%
annotator,,,,,,,,,,
MN,3.0,75.183729,75.759521,76.719175,78.318598,107.108207,156.332907,159.067612,160.708436,161.692930
RA,30.0,25.820410,26.619376,27.950986,30.376526,56.449169,155.708378,163.971816,167.403568,169.462619
both,33.0,25.829593,26.711211,28.180574,30.904432,58.582221,161.802318,163.380135,167.166895,169.438952


### PSO Features

In [24]:
pso_series = image_events_series[image_events_series.apply(lambda x: x.label == 3)]
ps_counts = pso_series.groupby(level="annotator").size().rename("total")

**DURATIONS**

In [25]:
pso_dur_percentiles = [
    pso_series.groupby(level="annotator").apply(
        lambda sub_series: sub_series.apply(lambda pso: pso.duration).quantile(q)
    ).rename(f"{100*q}%") for q in QUANTILES
]
pso_dur_percentiles = pd.concat([ps_counts, *pso_dur_percentiles], axis=1)
pso_dur_percentiles.loc["both"] = [
    ps_counts.sum(),
    *[pso_series.apply(lambda pso: pso.duration).quantile(q) for q in QUANTILES]
]

print("PURSUIT SMOOTHING DURATIONS (ms)")
print(f"{EVENT_PROPORTION}% have duration >= {pso_dur_percentiles.loc['both', '5.0%']:.2f}ms")
print(f"{EVENT_PROPORTION}% have duration <= {pso_dur_percentiles.loc['both', '95.0%']:.2f}ms")
print(f"{FEATURE_CI}% CI for duration is [{pso_dur_percentiles.loc['both', '2.5%']:.2f}, {pso_dur_percentiles.loc['both', '97.5%']:.2f}]ms\n")
pso_dur_percentiles

PURSUIT SMOOTHING DURATIONS (ms)
95% have duration >= 8.00ms
95% have duration <= 46.01ms
95% CI for duration is [6.00, 54.02]ms



,total,0.1%,1.0%,2.5%,5.0%,50.0%,95.0%,97.5%,99.0%,99.9%
annotator,,,,,,,,,,
MN,312.0,3.995244,5.99811,6.0000,7.9962,21.9990,50.00945,56.916825,64.45832,69.998689
RA,418.0,4.001000,4.99817,6.0047,7.9990,19.9985,40.01015,50.004025,55.00066,65.830583
both,730.0,3.996916,4.99829,6.0010,7.9990,20.0050,46.00855,54.020625,60.00000,69.999542


**MIN VELOCITIES**

In [26]:
pso_min_vel_percentiles = [
    pso_series.groupby(level="annotator").apply(
        lambda sub_series: sub_series.apply(lambda pso: pso.min_velocity).quantile(q)
    ).rename(f"{100*q}%") for q in QUANTILES
]
pso_min_vel_percentiles = pd.concat([ps_counts, *pso_min_vel_percentiles], axis=1)
pso_min_vel_percentiles.loc["both"] = [
    ps_counts.sum(),
    *[pso_series.apply(lambda pso: pso.min_velocity).quantile(q) for q in QUANTILES]
]

print("PSO MIN VELOCITIES (DVA/s)")
print(f"{EVENT_PROPORTION}% have min velocity >= {pso_min_vel_percentiles.loc['both', '5.0%']:.2f}°/s")
print(f"{EVENT_PROPORTION}% have min velocity <= {pso_min_vel_percentiles.loc['both', '95.0%']:.2f}°/s")
print(f"{FEATURE_CI}% CI for min velocity is [{pso_min_vel_percentiles.loc['both', '2.5%']:.2f}, {pso_min_vel_percentiles.loc['both', '97.5%']:.2f}]°/s\n")
pso_min_vel_percentiles

PSO MIN VELOCITIES (DVA/s)
95% have min velocity >= 3.06°/s
95% have min velocity <= 38.32°/s
95% CI for min velocity is [2.18, 45.07]°/s



,total,0.1%,1.0%,2.5%,5.0%,50.0%,95.0%,97.5%,99.0%,99.9%
annotator,,,,,,,,,,
MN,312.0,1.357170,1.811174,2.660823,3.481902,14.781779,43.233754,52.470840,63.514153,71.494832
RA,418.0,0.679208,1.611016,1.994167,2.682294,12.012524,34.239748,40.672586,46.224312,79.180612
both,730.0,0.773734,1.683227,2.176975,3.060359,13.242168,38.324552,45.069794,57.817433,76.091224


**PEAK VELOCITIES**

In [27]:
pso_peak_vel_percentiles = [
    pso_series.groupby(level="annotator").apply(
        lambda sub_series: sub_series.apply(lambda pso: pso.peak_velocity).quantile(q)
    ).rename(f"{100*q}%") for q in QUANTILES
]
pso_peak_vel_percentiles = pd.concat([ps_counts, *pso_peak_vel_percentiles], axis=1)
pso_peak_vel_percentiles.loc["both"] = [
    ps_counts.sum(),
    *[pso_series.apply(lambda pso: pso.peak_velocity).quantile(q) for q in QUANTILES]
]

print("PSO PEAK VELOCITIES (DVA/s)")
print(f"{EVENT_PROPORTION}% have peak velocity >= {pso_peak_vel_percentiles.loc['both', '5.0%']:.2f}°/s")
print(f"{EVENT_PROPORTION}% have peak velocity <= {pso_peak_vel_percentiles.loc['both', '95.0%']:.2f}°/s")
print(f"{FEATURE_CI}% CI for peak velocity is [{pso_peak_vel_percentiles.loc['both', '2.5%']:.2f}, {pso_peak_vel_percentiles.loc['both', '97.5%']:.2f}]°/s\n")
pso_peak_vel_percentiles

PSO PEAK VELOCITIES (DVA/s)
95% have peak velocity >= 36.62°/s
95% have peak velocity <= 132.69°/s
95% CI for peak velocity is [27.85, 138.45]°/s



,total,0.1%,1.0%,2.5%,5.0%,50.0%,95.0%,97.5%,99.0%,99.9%
annotator,,,,,,,,,,
MN,312.0,15.129122,24.571415,29.055168,38.320387,80.485172,133.968333,139.891068,144.061798,151.529737
RA,418.0,12.966291,21.302556,27.562203,34.597067,75.928541,129.441985,134.822194,141.532676,145.134590
both,730.0,12.727629,22.138687,27.845711,36.623740,78.286585,132.685562,138.449927,143.689045,148.663984


### Blink Features

In [28]:
bl_series = image_events_series[image_events_series.apply(lambda x: x.label == 5)]
bl_counts = bl_series.groupby(level="annotator").size().rename("total")

**DURATIONS**

In [29]:
bl_dur_percentiles = [
    bl_series.groupby(level="annotator").apply(
        lambda sub_series: sub_series.apply(lambda bl: bl.duration).quantile(q)
    ).rename(f"{100*q}%") for q in QUANTILES
]
bl_dur_percentiles = pd.concat([bl_counts, *bl_dur_percentiles], axis=1)
bl_dur_percentiles.loc["both"] = [
    bl_counts.sum(),
    *[bl_series.apply(lambda bl: bl.duration).quantile(q) for q in QUANTILES]
]

print("BLINK DURATIONS (ms)")
print(f"{EVENT_PROPORTION}% have duration >= {bl_dur_percentiles.loc['both', '5.0%']:.2f}ms")
print(f"{EVENT_PROPORTION}% have duration <= {bl_dur_percentiles.loc['both', '95.0%']:.2f}ms")
print(f"{FEATURE_CI}% CI for duration is [{bl_dur_percentiles.loc['both', '2.5%']:.2f}, {bl_dur_percentiles.loc['both', '97.5%']:.2f}]ms\n")
bl_dur_percentiles

BLINK DURATIONS (ms)
95% have duration >= 95.01ms
95% have duration <= 617.34ms
95% CI for duration is [94.12, 633.74]ms



,total,0.1%,1.0%,2.5%,5.0%,50.0%,95.0%,97.5%,99.0%,99.9%
annotator,,,,,,,,,,
MN,22.0,90.129727,91.07227,92.643175,97.8626,319.069,520.0189,574.2766,610.19164,631.740664
RA,23.0,94.046714,94.24214,94.567850,99.5140,329.996,623.7379,839.8858,993.70012,1085.988712
both,45.0,90.201000,91.78500,94.123700,95.0120,322.071,617.3368,633.7354,892.91548,1075.910248
